In [40]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timezone
import os
from dotenv import load_dotenv

# Load environment variables and connect to DB
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

# Load primary data
technical_df = pd.read_sql("SELECT * FROM technical_features", engine, parse_dates=["date"])
sentiment_df = pd.read_sql("SELECT * FROM sentiment_data", engine, parse_dates=["date"])

# Optional data loads
try:
    macro_df = pd.read_sql("SELECT * FROM macro_data", engine, parse_dates=["date"])
except Exception as e:
    print(f"⚠️ Skipping macro_data: {e}")
    macro_df = pd.DataFrame()

try:
    fundamentals_df = pd.read_sql("SELECT * FROM fundamental_data", engine, parse_dates=["date"])
except Exception as e:
    print(f"⚠️ Skipping fundamental_data: {e}")
    fundamentals_df = pd.DataFrame()
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timezone
import os
from dotenv import load_dotenv

# Load .env
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

# Load technical and sentiment
technical_df = pd.read_sql("SELECT * FROM technical_features", engine, parse_dates=["date"])
sentiment_df = pd.read_sql("SELECT * FROM sentiment_data", engine, parse_dates=["date"])

# Optional macro + fundamentals
try:
    macro_df = pd.read_sql("SELECT * FROM macro_data", engine, parse_dates=["date"])
except Exception as e:
    print(f"⚠️ Skipping macro_data: {e}")
    macro_df = pd.DataFrame()

try:
    fundamentals_df = pd.read_sql("SELECT * FROM fundamental_data", engine, parse_dates=["date"])
except Exception as e:
    print(f"⚠️ Skipping fundamental_data: {e}")
    fundamentals_df = pd.DataFrame()

# Aggregate sentiment
sentiment_agg = sentiment_df.groupby(["ticker", "date"]).agg(
    sentiment_avg=("sentiment_score", "mean"),
    sentiment_std=("sentiment_score", "std"),
    sentiment_count=("sentiment_score", "count")
).reset_index()

# Merge technical + sentiment
merged = pd.merge(technical_df, sentiment_agg, on=["ticker", "date"], how="left")
merged = merged.sort_values(["ticker", "date"])

# 🔁 Merge most recent fundamental data per ticker
if not fundamentals_df.empty:
    fundamentals_df = fundamentals_df.sort_values(["symbol", "date"])
    if "ticker" not in fundamentals_df.columns:
        fundamentals_df.rename(columns={"symbol": "ticker"}, inplace=True)

    fundamentals_df.columns.name = None  # flatten any pivoted column names
    merged = pd.merge_asof(
        merged,
        fundamentals_df,
        by="ticker",
        on="date",
        direction="backward",
        allow_exact_matches=True
    )

# 🔁 Merge most recent macro data
if not macro_df.empty:
    macro_df = macro_df.sort_values("date")
    merged = pd.merge_asof(
        merged,
        macro_df,
        on="date",
        direction="backward",
        allow_exact_matches=True
    )

# Final metadata
merged["merged_at"] = datetime.now(timezone.utc)

# Save to DB
merged.to_sql("merged_features", engine, if_exists="replace", index=False)
print(f"✅ Merged features table created with {len(merged)} rows.")


✅ Merged features table created with 6641 rows.


In [41]:
merged

,ticker,date,close,volume,return_1d,sma_5,sma_20,ema_10,rsi_14,macd,...,payablesTurnover,inventoryTurnover,roe,capexPerShare,fetched_at_x,interest_rate,cpi,unemployment,fetched_at_y,merged_at
0,NVDA,1999-01-22,0.041016,2714688000,NaN,NaN,NaN,0.041016,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaT,4.63,164.300,4.3,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
1,NVDA,1999-01-25,0.045313,510480000,0.104764,NaN,NaN,0.041797,NaN,0.000343,...,NaN,NaN,NaN,NaN,NaT,4.63,164.300,4.3,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
2,NVDA,1999-01-26,0.041797,343200000,-0.077594,NaN,NaN,0.041797,NaN,0.000327,...,NaN,NaN,NaN,NaN,NaT,4.63,164.300,4.3,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
3,NVDA,1999-01-27,0.041667,244368000,-0.003110,NaN,NaN,0.041774,NaN,0.000300,...,NaN,NaN,NaN,NaN,NaT,4.63,164.300,4.3,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
4,NVDA,1999-01-28,0.041536,227520000,-0.003144,0.042266,NaN,0.041730,NaN,0.000266,...,NaN,NaN,NaN,NaN,NaT,4.63,164.300,4.3,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6636,NVDA,2025-06-10,143.960007,155881900,0.009325,142.044003,136.741001,139.823866,65.362411,6.274724,...,5.172583,3.237996,0.918729,0.131786,2025-06-18 05:39:52.094010,4.33,321.465,4.2,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
6637,NVDA,2025-06-11,142.830002,167694000,-0.007849,142.226004,137.386002,140.370436,68.550277,6.123080,...,5.172583,3.237996,0.918729,0.131786,2025-06-18 05:39:52.094010,4.33,321.465,4.2,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
6638,NVDA,2025-06-12,145.000000,162365000,0.015193,143.228003,137.869002,141.212175,69.711685,6.107597,...,5.172583,3.237996,0.918729,0.131786,2025-06-18 05:39:52.094010,4.33,321.465,4.2,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
6639,NVDA,2025-06-13,141.970001,180820600,-0.020897,143.278003,138.226002,141.349962,66.501866,5.784155,...,5.172583,3.237996,0.918729,0.131786,2025-06-18 05:39:52.094010,4.33,321.465,4.2,2025-06-18 05:40:00.775113,2025-06-18 10:40:35.760899+00:00
